# Backend API

Wrap the following APIs using Python requests:

| Method | URL | Description |
| --- | --- | --- |
| GET | `http://{{host}}/backend-api/accounts/check` | check account status |
| GET | `http://{{host}}/backend-api/models?history_and_training_disabled=false` | get models |
| GET | `http://{{host}}/backend-api/settings/beta_features` | get beta features |
| GET | `http://{{host}}/public-api/conversation_limit` | get conversation limit(GPT4) |
| GET | `http://{{host}}/backend-api/conversations?offset=0&limit=3&order=updated` | get conversation list |
| GET | `http://{{host}}/backend-api/conversation/{{conversation_id}}` | get conversation by id |
| PATCH | `http://{{host}}/backend-api/conversation/{{conversation_id}}` | clear conversation by id |
| POST | `http://{{host}}/backend-api/conversation` | new conversation |
| GET | `http://{{host}}/backend-api/shared_conversations?order=created` | get share link |
| POST | `http://{{host}}/backend-api/accounts/data_export` | export data send to email |
| PATCH | `http://{{host}}/backend-api/conversation/{{conversation_id}}` | change conversation title by id |
| POST | `http://{{host}}/backend-api/conversation/gen_title/{{conversation_id}}` | generate conversation title |

## Setup

```bash
export OPENAI_API_BASE_URL="https://ninja.yourdomain.com"
export BACKEND_REVERSE_PROXY="https://ninja.yourdomain.com/backend-api"
export OPENAI_ACCESS_TOKEN="your_openai_access_token"
```


## Initialize

In [1]:
import os, json, requests, uuid
from pprint import pprint
from typing import Union
# set default values
base_url = os.getenv("OPENAI_API_BASE_URL")
backend_url = os.getenv("OPENAI_API_BACKEND_URL")
access_token = os.getenv("OPENAI_API_KEY")

## API Wrapper

### Check account status

In [2]:
def get_account_status(backend_url:str, access_token:str):
    """Get account status from backend-api

    Args:
        backend_url (str): backend url
        access_token (str): access token at https://chat.openai.com/api/auth/session
    
    Returns:
        dict: account status
    
    Rest API:
        ### check account status
        GET http://{{host}}/backend-api/accounts/check
        Authorization: {{bearer_token}}
    """
    url = os.path.join(backend_url, "accounts/check")
    headers = {
        'Content-Type': 'application/json',
        'Authorization': f'Bearer {access_token}'
    }
    response = requests.get(url, headers=headers)
    return response.json()

## Example
resp = get_account_status(backend_url, access_token)
pprint(resp)

{'account_plan': {'account_user_role': 'account-owner',
                  'has_customer_object': False,
                  'is_paid_subscription_active': False,
                  'subscription_expires_at_timestamp': 1702560866,
                  'subscription_plan': 'chatgptfreeplan',
                  'was_paid_customer': True},
 'features': ['chat_preferences_available',
              'shareable_links',
              'allow_url_thread_creation',
              'disable_team_upgrade_ui',
              'user_settings_announcements',
              'targeted_replies',
              'bizmo_settings',
              'arkose_gpt_35_experiment',
              'breeze_available',
              'gizmo_live',
              'privacy_policy_nov_2023',
              'starter_prompts',
              'conversation_bot_arkose',
              'new_plugin_oauth_endpoint',
              'gizmo_ui',
              'check_age_verification',
              'invite_referral',
              'arkose_enabled'],
 'u

### Get models

In [3]:
def get_models(backend_url:str, access_token:str, history_and_training_disabled:bool=False):
    """Get models from backend-api

    Args:
        backend_url (str): backend url
        access_token (str): access token at https://chat.openai.com/api/auth/session
        history_and_training_disabled (bool, optional): history and training disabled. Defaults to False.
    
    Returns:
        dict: models
    
    Rest API:
        ### get models
        GET http://{{host}}/backend-api/models?history_and_training_disabled=false
        Authorization: {{bearer_token}}
    """
    url = os.path.join(backend_url, "models")
    params = {
        "history_and_training_disabled": history_and_training_disabled
    }
    headers = {
        'Content-Type': 'application/json',
        'Authorization': f'Bearer {access_token}'
    }
    response = requests.get(url, params=params, headers=headers)
    return response.json()

## Example
resp = get_models(backend_url, access_token)
print([model['category'] for model in resp["categories"]])
pprint(resp)

['gpt_3.5']
{'categories': [{'category': 'gpt_3.5',
                 'code_interpreter_model': 'text-davinci-002-render-sha-code-interpreter',
                 'default_model': 'text-davinci-002-render-sha',
                 'human_category_name': 'GPT-3.5',
                 'plugins_model': 'text-davinci-002-render-sha-plugins',
                 'subscription_level': 'free'}],
 'models': [{'capabilities': {},
             'description': 'Our fastest model, great for most everyday tasks.',
             'max_tokens': 8191,
             'product_features': {},
             'slug': 'text-davinci-002-render-sha',
             'tags': ['gpt3.5'],
             'title': 'Default (GPT-3.5)'}]}


### Get Beta Features

In [4]:
def get_beta_features(backend_url:str, access_token:str):
    """Get beta features from backend-api

    Args:
        backend_url (str): backend url
        access_token (str): access token at https://chat.openai.com/api/auth/session
    
    Returns:
        dict: beta features
    
    Rest API:
        ### get beta features
        GET http://{{host}}/backend-api/settings/beta_features
        Authorization: {{bearer_token}}
    """
    url = os.path.join(backend_url, "settings/beta_features")
    headers = {
        'Content-Type': 'application/json',
        'Authorization': f'Bearer {access_token}'
    }
    response = requests.get(url, headers=headers)
    return response.json()

## Example
pprint(get_beta_features(backend_url, access_token))

{'browsing': True,
 'code_interpreter': False,
 'dalle': False,
 'plugins': False,
 'sunshine': False,
 'workspace_gpt_custom_actions': False}


### Get Chat List

In [5]:
def get_chat_list( backend_url:str, access_token:str
                 , offset:int=0, limit:Union[int, None]=None, order:str="updated"):
    """Get chat list from backend-api

    Args:
        backend_url (str): backend url
        access_token (str): access token at https://chat.openai.com/api/auth/session
        offset (int, optional): start index. Defaults to 0.
        limit (int, optional): max number of chat. Defaults to 3.
        order (str, optional): order by. Defaults to "updated".
    
    Returns:
        dict: chat list
    
    Rest API:
        ### get conversation list
        GET http://{{host}}/backend-api/conversations?offset=0&limit=3&order=updated
        Authorization: {{bearer_token}}
    """
    url = os.path.join(backend_url, "conversations")
    params = {
        "offset": offset,
        "limit": limit,
        "order": order
    }
    headers = {
        'Content-Type': 'application/json',
        'Authorization': f'Bearer {access_token}'
    }
    response = requests.get(url, params=params, headers=headers)
    return response.json()
## example
resp = get_chat_list(backend_url, access_token, limit=1)
pprint([{'conversation_id':item['id'],'title': item['title']} for item in resp['items']])
pprint(resp)

[{'conversation_id': 'be06ea4d-a482-49ee-93a8-3dbb68b39f74',
  'title': 'User Request: Summarize Conversation'}]
{'has_missing_conversations': False,
 'items': [{'conversation_template_id': None,
            'create_time': '2023-12-22T03:46:06.392671+00:00',
            'current_node': None,
            'gizmo_id': None,
            'id': 'be06ea4d-a482-49ee-93a8-3dbb68b39f74',
            'is_archived': False,
            'mapping': None,
            'title': 'User Request: Summarize Conversation',
            'update_time': '2023-12-22T03:54:24.776292+00:00',
            'workspace_id': None}],
 'limit': 1,
 'offset': 0,
 'total': 1635}


### Chat Completion

In [7]:
def chat_completion( backend_url:str, access_token:str
                   , prompt:str
                   , current_message_id:str
                   , parent_message_id:str
                   , conversation_id:Union[str, None]=None
                   , model:str="text-davinci-002-render-sha"
                   , history_and_training_disabled:bool=False):
    """chat completion(create or edit)

    Args:
        backend_url (str): backend url
        access_token (str): access token at https://chat.openai.com/api/auth/session
        prompt (str): prompt
        current_message_id (str): current message id. You can use uuid.uuid4() to generate one.
        parent_message_id (str): parent message id.
        conversation_id (str, optional): conversation id. Defaults to None.
        model (str, optional): model. Defaults to "text-davinci-002-render-sha".
        history_and_training_disabled (bool, optional): disable history record in the website. Defaults to False.
    
    Returns:
        dict: chat
    
    Rest API:
        ### new conversation
        POST http://{{host}}/backend-api/conversation
        Authorization: {{bearer_token}}
        Content-Type: application/json
        Accept: text/event-stream
    
    {
        "action": "next",
        "messages": [
            {
            "id": "{{$guid}}",
            "author": {
                "role": "user"
            },
            "content": {
                "content_type": "text",
                "parts": [
                "new conversation"
                ]
            },
            "metadata": {}
            }
        ],
        "model": "text-davinci-002-render-sha-mobile",
        "parent_message_id": "{{$guid}}",
        "timezone_offset_min": -480,
        "history_and_training_disabled": false
    }
    """
    url = os.path.join(backend_url, "conversation")
    headers = {
        'Content-Type': 'application/json',
        'Authorization': f'Bearer {access_token}',
        'Accept': 'text/event-stream'
    }
    messages = [
        {
            "id": current_message_id,
            "author": {"role": "user"},
            "content": {"content_type": "text", "parts": [prompt]},
        },
    ]
    data = {
        "action": "next",
        "messages": messages,
        "conversation_id": conversation_id,
        "parent_message_id": parent_message_id,
        "model": model,
        "history_and_training_disabled": history_and_training_disabled,
    }
    response = requests.post(url, headers=headers, data=json.dumps(data))
    resps = response.text.split("data:")[:-1] # the last one is [DONE]
    return [json.loads(resp) for resp in resps if resp.strip() != ""]

In [8]:
## Example 1: create chat
parent_message_id, current_message_id = str(uuid.uuid4()), str(uuid.uuid4())
prompt = "hello"
resp = chat_completion(backend_url, access_token, prompt, parent_message_id, current_message_id)
conversation_id, message_id = resp[-1]['conversation_id'], resp[-1]['message_id']
message = resp[-2]['message']['content']['parts'][0]
## Note: you can continue/regenerate the response base on the parent_message_id
## Example 2: continue the chat
newprompt = "how are you?"
parent_message_id, current_message_id = message_id, str(uuid.uuid4())
newresp = chat_completion(backend_url, access_token, newprompt, parent_message_id, current_message_id, conversation_id)
newmessage = newresp[-2]['message']['content']['parts'][0]

print(f"prompt: {prompt}\nresponse: {message}\nprompt: {newprompt}\nresponse: {newmessage}")
print(f"conversation_id: {conversation_id}")
# The message is too long, so we only print the last two messages here
pprint(resp[-1])
pprint(newresp[-2])

prompt: hello
response: Hello! How can I assist you today?
prompt: how are you?
response: Hello! I'm just a computer program, so I don't have feelings, but I'm here and ready to help you with any questions or information you might need. How can I assist you today?
conversation_id: 9edf4b4b-3c43-4002-a666-c8e1493f4935
{'conversation_id': '9edf4b4b-3c43-4002-a666-c8e1493f4935',
 'is_completion': True,
 'message_id': '3a9186f9-1fbc-444a-ab13-9c2f2012de54',
 'moderation_response': {'blocked': False,
                         'flagged': False,
                         'moderation_id': 'modr-8YR2sJrIvr9OemnRoObGCb29GdrOg'}}
{'conversation_id': '9edf4b4b-3c43-4002-a666-c8e1493f4935',
 'error': None,
 'message': {'author': {'metadata': {}, 'name': None, 'role': 'assistant'},
             'content': {'content_type': 'text',
                         'parts': ["Hello! I'm just a computer program, so I "
                                   "don't have feelings, but I'm here and "
                   

### Chat By ID

In [9]:
def get_chat_by_id(backend_url:str, access_token:str, conversation_id:str):
    """Get chat by id from backend-api

    Args:
        backend_url (str): backend url
        access_token (str): access token at https://chat.openai.com/api/auth/session
        conversation_id (str): conversation id
    
    Returns:
        dict: chat
    
    Rest API:
        ### get conversation by id
        GET http://{{host}}/backend-api/conversation/5ae8355a-82a8-4ded-b0e4-ea5dc11b4a9f
        Authorization: {{bearer_token}}
    """
    url = os.path.join(backend_url, "conversation", conversation_id)
    headers = {
        'Content-Type': 'application/json',
        'Authorization': f'Bearer {access_token}'
    }
    response = requests.get(url, headers=headers)
    return response.json()

## Example
chat_list_resp = get_chat_list(backend_url, access_token, limit=1)
conversation_id = chat_list_resp['items'][0]['id']
resp = get_chat_by_id(backend_url, access_token, conversation_id)
pprint(resp)

{'conversation_id': '9edf4b4b-3c43-4002-a666-c8e1493f4935',
 'create_time': 1703217806.360512,
 'current_node': '3fc222d4-b78d-4634-b903-e7de75a54470',
 'is_archived': False,
 'mapping': {'3375543a-2b45-4eb3-b325-54307e3d3bf5': {'children': ['bd768a32-0124-418a-864d-146940e8fe4c'],
                                                      'id': '3375543a-2b45-4eb3-b325-54307e3d3bf5',
                                                      'message': {'author': {'metadata': {},
                                                                             'role': 'system'},
                                                                  'content': {'content_type': 'text',
                                                                              'parts': ['']},
                                                                  'end_turn': True,
                                                                  'id': '3375543a-2b45-4eb3-b325-54307e3d3bf5',
                                    

### Delete Chat

In [10]:
def delete_chat(backend_url:str, access_token:str, conversation_id:str):
    """Delete chat by id

    Args:
        backend_url (str): backend url
        access_token (str): access token at https://chat.openai.com/api/auth/session
        conversation_id (str): conversation id
    
    Returns:
        dict: response
    
    Rest API:
        ### clear conversation by id
        PATCH http://{{host}}/backend-api/conversation/5ae8355a-82a8-4ded-b0e4-ea5dc11b4a9f
        Authorization: {{bearer_token}}
        Content-Type: application/json

        {
            "is_visible": false
        }
    """
    url = os.path.join(backend_url, "conversation", conversation_id)
    headers = {
        'Content-Type': 'application/json',
        'Authorization': f'Bearer {access_token}'
    }
    data = {
        "is_visible": False
    }
    response = requests.patch(url, headers=headers, data=json.dumps(data))
    return response.json()

## Example
chat_list_resp = get_chat_list(backend_url, access_token, limit=1)
conversation_id = chat_list_resp['items'][0]['id']
resp = delete_chat(backend_url, access_token, conversation_id)
pprint(resp)

{'success': True}


### Get Shared Links

In [17]:
def get_share_links(backend_url:str, access_token:str, order:str="created"):
    """Get share links from backend-api

    Args:
        backend_url (str): backend url
        access_token (str): access token at https://chat.openai.com/api/auth/session
        order (str, optional): order by. Defaults to "created".
    
    Returns:
        dict: share links
    
    Rest API:
        ### get share link
        GET http://{{host}}/backend-api/shared_conversations?order=created
        Authorization: {{bearer_token}}
    """
    url = os.path.join(backend_url, "shared_conversations")
    params = {
        "order": order
    }
    headers = {
        'Content-Type': 'application/json',
        'Authorization': f'Bearer {access_token}'
    }
    response = requests.get(url, params=params, headers=headers)
    return response.json()

## xample
resp = get_share_links(backend_url, access_token)
# pprint(resp) # too long, so we only print the first one
resp['items'] = resp['items'][:1]
pprint(resp)

{'has_missing_conversations': False,
 'items': [{'conversation_id': '6677a2e1-bb24-45a0-8f81-8609e4a53665',
            'conversation_template_id': None,
            'create_time': '2023-12-13T17:16:53.407544+00:00',
            'current_node': None,
            'gizmo_id': None,
            'id': '9942703e-0b2b-42b3-aee5-ed0de8335e2d',
            'is_archived': None,
            'mapping': None,
            'title': '物流优化问题解决方案',
            'update_time': '2023-12-13T17:17:10+00:00',
            'workspace_id': None}],
 'limit': 50,
 'offset': 0,
 'total': 14}


### Edit Chat Title

In [12]:
def edit_chat_title(backend_url:str, access_token:str, conversation_id:str, title:str):
    """Edit chat title by id

    Args:
        backend_url (str): backend url
        access_token (str): access token at https://chat.openai.com/api/auth/session
        conversation_id (str): conversation id
        title (str): title
    
    Returns:
        dict: response
    
    Rest API:
        ### change conversation title by id
        PATCH http://{{host}}/backend-api/conversation/5ae8355a-82a8-4ded-b0e4-ea5dc11b4a9f
        Authorization: {{bearer_token}}
        Content-Type: application/json

        {
            "title": "New Test"
        }
    """
    url = os.path.join(backend_url, "conversation", conversation_id)
    headers = {
        'Content-Type': 'application/json',
        'Authorization': f'Bearer {access_token}'
    }
    data = {
        "title": title
    }
    response = requests.patch(url, headers=headers, data=json.dumps(data))
    return response.json()

## Example
chat_list_resp = get_chat_list(backend_url, access_token, limit=1)
conversation_id = chat_list_resp['items'][0]['id']
resp = edit_chat_title(backend_url, access_token, conversation_id, "New Test")
pprint(resp)

{'success': True}


### Generate Chat Title

In [15]:
def generate_chat_title(backend_url:str, access_token:str, conversation_id:str, message_id:str):
    """Generate chat title by id and message id

    Args:
        backend_url (str): backend url
        access_token (str): access token at https://chat.openai.com/api/auth/session
        conversation_id (str): conversation id
        message_id (str): message id
    
    Returns:
        dict: response
    
    Rest API:
        ### generate conversation title
        POST http://{{host}}/backend-api/conversation/gen_title/5ae8355a-82a8-4ded-b0e4-ea5dc11b4a9f
        Authorization: {{bearer_token}}
        Content-Type: application/json

        {
            "message_id": "1646facc-08a6-465f-ba08-58cec1e31ed6"
        }
    """
    url = os.path.join(backend_url, "conversation/gen_title", conversation_id)
    headers = {
        'Content-Type': 'application/json',
        'Authorization': f'Bearer {access_token}'
    }
    data = {
        "message_id": message_id
    }
    response = requests.post(url, headers=headers, data=json.dumps(data))
    return response.json()

## Example
chat_list_resp = get_chat_list(backend_url, access_token, limit=1)
conversation_id = chat_list_resp['items'][0]['id']
resp = get_chat_by_id(backend_url, access_token, conversation_id)
message_id = resp['current_node']
resp = generate_chat_title(backend_url, access_token, conversation_id, message_id)
pprint(resp)

{'message': 'Conversation be06ea4d-a482-49ee-93a8-3dbb68b39f74 already has '
            "title 'New Test'"}


### Get Conversation Limit

In [16]:
def get_conversation_limit(base_url:str, access_token:str):
    """Get conversation limit of gpt-4

    Args:
        base_url (str): base url
        access_token (str): access token at https://chat.openai.com/api/auth/session
    
    Returns:
        dict: conversation limit
    
    Rest API:
        ### get conversation limit
        GET http://{{host}}/public-api/conversation_limit
        Authorization: {{bearer_token}}
    """
    url = os.path.join(base_url, "public-api/conversation_limit")
    headers = {
        'Content-Type': 'application/json',
        'Authorization': f'Bearer {access_token}'
    }
    response = requests.get(url, headers=headers)
    return response.json()

## Example
resp = get_conversation_limit(base_url, access_token)
pprint(resp)

{'message_cap': 40.0,
 'message_cap_window': 180.0,
 'message_disclaimer': {'model-switcher': "You've reached the GPT-4 cap, which "
                                          'gives all ChatGPT Plus users a '
                                          'chance to try the model.\n'
                                          '\n'
                                          'Please check back soon.',
                        'textarea': 'GPT-4 currently has a cap of 40 messages '
                                    'every 3 hours.'}}


### Send Data to Email

In [43]:
def send_data_to_email(backend_url:str, access_token:str):
    """Send data to email

    Args:
        backend_url (str): backend url
        access_token (str): access token at https://chat.openai.com/api/auth/session
    
    Returns:
        dict: response
    
    Rest API:
        ### export data send to email
        POST http://{{host}}/backend-api/accounts/data_export
        Authorization: {{bearer_token}}
    """
    url = os.path.join(backend_url, "accounts/data_export")
    headers = {
        'Content-Type': 'application/json',
        'Authorization': f'Bearer {access_token}'
    }
    response = requests.post(url, headers=headers)
    return response.json()

## Example
resp = send_data_to_email(backend_url, access_token)
pprint(resp)

{'status': 'queued'}
